In [0]:
import pyspark
from pyspark.sql import SparkSession


In [0]:
spark=SparkSession.builder.appName("Clustering").getOrCreate()

In [0]:
from pyspark.ml.clustering import KMeans
df=spark.read.csv("dbfs:/FileStore/shared_uploads/purvajainpj123@gmail.com/Hack_data-2.csv",inferSchema=True,header=True)

In [0]:
df.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [0]:
df.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

In [0]:
df.columns

Out[8]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [0]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [0]:
feat_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
             'Servers_Corrupted', 'Pages_Corrupted','WPM_Typing_Speed']

In [0]:
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')

In [0]:
final_data = vec_assembler.transform(df)

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [0]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

In [0]:
cluster_final_data = scalerModel.transform(final_data)

In [0]:
# find out whether there are 2 or 3 hackers
kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)

In [0]:
model_k3 = kmeans3.fit(cluster_final_data)
model_k2 = kmeans2.fit(cluster_final_data)

In [0]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator()
wssse_k3 = evaluator.evaluate(model_k3.transform(cluster_final_data))
wssse_k2 = evaluator.evaluate(model_k2.transform(cluster_final_data))


In [0]:
print("With K=3")
print("Within Set Sum of Squared Errors = " + str(wssse_k3))
print('--'*30)
print("With K=2")
print("Within Set Sum of Squared Errors = " + str(wssse_k2))

With K=3
Within Set Sum of Squared Errors = 0.3068084951287429
------------------------------------------------------------
With K=2
Within Set Sum of Squared Errors = 0.6683623593283755


In [0]:
for k in range(2,9):
    kmeans = KMeans(featuresCol='scaledFeatures',k=k)
    model = kmeans.fit(cluster_final_data)
    wssse = evaluator.evaluate(model.transform(cluster_final_data))
    print("With K={}".format(k))
    print("Within Set Sum of Squared Errors = " + str(wssse))
    print('--'*30)

With K=2
Within Set Sum of Squared Errors = 0.6683623593283755
------------------------------------------------------------
With K=3
Within Set Sum of Squared Errors = 0.3068084951287429
------------------------------------------------------------
With K=4
Within Set Sum of Squared Errors = -0.04792891045570489
------------------------------------------------------------
With K=5
Within Set Sum of Squared Errors = -0.10219656272982447
------------------------------------------------------------
With K=6
Within Set Sum of Squared Errors = -0.14728200598045754
------------------------------------------------------------
With K=7
Within Set Sum of Squared Errors = -0.16032422559982856
------------------------------------------------------------
With K=8
Within Set Sum of Squared Errors = -0.16306925639719894
------------------------------------------------------------


In [0]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   84|
|         0|  167|
+----------+-----+



In [0]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

